### [Construct Binary Tree from in-order and post-order traversal](https://leetcode.com/problems/construct-binary-tree-from-inorder-and-postorder-traversal/description/)

Given inorder and postorder traversal of a tree, construct the binary tree.

Note:
You may assume that duplicates do not exist in the tree.


For example, given
```

inorder = [9,3,15,20,7]
postorder = [9,15,7,20,3]
Return the following binary tree:

    3
   / \
  9  20
    /  \
   15   7
```

In [2]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Solution:
    
    def buildTree(self, inOrder, postOrder):
        """
        :type inorder: List[int]
        :type postorder: List[int]
        :rtype: TreeNode
        """
        
        
        def buildTreeWorker(inOrder, postOrder, start, end, inOrderMap):
            """
            :type inOrder: List[int]
            :type postOrder: List[int]
            :type start: int
            :type end: int
            :type inOrderMap: dict
            :rtype: TreeNode
            """

            # Refer to V1 for my intuition
            # this one has minor optimizations to avoid the cost of O(n) lookup
            # in every iteration.

            if not inOrder or not postOrder:
                return None
            
            # equivalent of running out of elements in the inOrder list
            # Need >= because I pass len(inOrder) in the first call
            if start >= end:
                return None

            # since we pickup the last element in the postOrder as the root
            # in every iteration, we don't have to worry about finding the
            # boundary inside the post order. instead, just pop the last element
            # that would take care of the order. But remember: popping the element
            # will alter the input

            root = TreeNode(postOrder.pop())
            rootIndex = inOrderMap[root.val]

            # Order of right and left is very important. Because in post order
            # traversal, the node adjacent to the root is the root of right subtree
            # so we build the root nodes from the postOrder, we have to build the
            # right tree first after building the current root.
            root.right = buildTreeWorker(inOrder, postOrder, rootIndex+1, end, inOrderMap)
            root.left = buildTreeWorker(inOrder, postOrder, start, rootIndex, inOrderMap)

            return root
        
        
        # Stay paranoid.
        if not inOrder or not postOrder:
                return None
        
        # Take a one time hit of O(n) to build the map
        inOrderMap = {
            val : index for index, val in enumerate(inOrder)
        }
        
        return buildTreeWorker(inOrder, postOrder, 0, len(inOrder), inOrderMap)
    
    def buildTreeV2(self, inOrder, postOrder):
        """
        :type inorder: List[int]
        :type postorder: List[int]
        :rtype: TreeNode
        """
        # Refer to V1 for my intuition
        # this one has minor optimizations.
        
        if not inOrder or not postOrder:
            return None
        
        # since we pickup the last element in the postOrder as the root
        # in every iteration, we don't have to worry about finding the
        # boundary inside the post order. instead, just pop the last element
        # that would take care of the order. But remember: popping the element
        # will alter the input
        
        root = TreeNode(postOrder.pop())
        rootIndex = inOrder.index(root.val)
        
        # Order of right and left is very important. Because in post order
        # traversal, the node adjacent to the root is the root of right subtree
        # so we build the root nodes from the postOrder, we have to build the
        # right tree first after building the current root.
        root.right = self.buildTree(inOrder[rootIndex + 1:], postOrder)
        root.left = self.buildTree(inOrder[:rootIndex], postOrder)
        
        return root
        
        
    def buildTreeV1(self, inOrder, postOrder):
        """
        :type inorder: List[int]
        :type postorder: List[int]
        :rtype: TreeNode
        """
        # root index = last element in my postOrder
        # lookup root in the inOrder list, find the index (no duplicates.. so no worries about conflicts)
        # 0..root_index : left of root, rootIndex + 1 : endOfInOrder = right of root, in inOrder list
        # leftLength = rootIndex
        # rightLength = len(inOrder) - rootIndex - 1
        # we can reconstruct the inOrder and postOrder for the left and right subtree
        # careful about indexing.. make sure no out of bounds acccess
        
        # edge cases
        # at any given time, length of inOrder and postOrder should match
        if not inOrder or not postOrder:
            return None
        
        root = TreeNode(postOrder[-1])
        rootIndex = inOrder.index(root.val)
        leftLength = rootIndex
        rightLength = len(inOrder) - rootIndex - 1
        
        root.left = self.buildTree(inOrder[:leftLength], postOrder[:leftLength])
        root.right = self.buildTree(inOrder[rootIndex + 1:], postOrder[leftLength: leftLength + rightLength])
        
        return root
        

In [10]:
# Test inputs
# [2, 5, 9, 10, 12, 15, 20]
# [2, 9, 5, 12, 20, 15, 10]

# [2, 5, 9, 15]
# [2, 9, 5, 15]

# Call buildTree() on the test inputs. Do an inorder traversal of the returned Tree and verify
# that the inorder traversal of the built tree matches with the given inorder traversal

class TestInput:
    def __init__(self, inOrder, postOrder):
        self.inOrder = inOrder
        self.postOrder = postOrder

def inOrderGen(root):
    if root:
        yield from inOrderGen(root.left)
        yield root.val
        yield from inOrderGen(root.right)

testInputs = []

def init(testInputs):
    testInputs.clear()
    testInputs.append(TestInput([2, 5, 9, 10, 12, 15, 20], [2, 9, 5, 12, 20, 15, 10]))
    testInputs.append(TestInput([2, 5, 9, 15], [2, 9, 5, 15]))
    testInputs.append(TestInput([9,3,15,20,7], [9,15,7,20,3]))

def testDefault():
    s = Solution()
    init(testInputs)
    for testInput in testInputs:
        generatedTree = s.buildTree(testInput.inOrder, testInput.postOrder)
        assert testInput.inOrder == list(inOrderGen(generatedTree))

def testV2():
    s = Solution()
    init(testInputs)
    for testInput in testInputs:
        generatedTree = s.buildTreeV2(testInput.inOrder, testInput.postOrder)
        assert testInput.inOrder == list(inOrderGen(generatedTree))

def testV1():
    s = Solution()
    init(testInputs)
    for testInput in testInputs:
        generatedTree = s.buildTreeV1(testInput.inOrder, testInput.postOrder)
        assert testInput.inOrder == list(inOrderGen(generatedTree))

%timeit testDefault()
%timeit testV2()
%timeit testV1()

# The different solutions didn't make much difference with small inputs.
# but it made a big difference with large inputs as O(n^2) is reduced to
# O(n) with the help of additional space. The optimized one ran in 56ms
# whereas the non-optimized ones took about 200ms with the leetcode test.

44.3 µs ± 4.55 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
48.5 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
46.9 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
